### Functions that were made during part 3 to keep track of balances across various bookmakers accounts.

In [ ]:
import pandas as pd

In [17]:
bet_spreadsheet = pd.read_csv('Bet Spreadsheet with profit features.csv')
bet_spreadsheet.head(5)

,Date Created,Sport,Event,Event Time,Bookie,Bet Type,Type,Outcome,Stake,Odds,Fee (%),Liability,Return,Potential Profit,Bet Result,Profit,Running Profit,Note,Profit ID
0,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Betfred,Qualifying,Back,Leicester,10.00,2.05,0,0.00,10.50,-0.43,Win,-0.43,-0.43,,7967100
1,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Smarkets,Qualifying,Lay,Leicester,9.76,2.12,2,10.93,9.56,-0.44,Lose,0.00,-0.43,,7967100
2,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Coral,Qualifying,Back,Rochdale,5.00,1.91,0,0.00,4.55,-0.10,Win,2.57,2.14,2.67 of losses from exchange refunded,7967200
3,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,Rochdale,5.00,1.93,2,4.65,4.90,-0.10,Lose,0.00,2.14,2.67 of losses from exchange refunded,7967200
4,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Coral,Free (SNR),Back,Hungary,20.00,6.50,0,0.00,110.00,15.92,Win,15.92,18.06,,7967346


#### This was my original way of calculating the balances of each bookmaker for the qualifying bets.

#### The process of creating bookie_index_list was generalised and became the value_index_list() function, which may be useful in the future.

In [9]:
winning_qualifying_bets = (bet_spreadsheet['Bet Type'] == 'Qualifying')   \
                     & (bet_spreadsheet['Bet Result'] == 'Win')

bookie_index_list = {}
for index, bookie in bet_spreadsheet[winning_qualifying_bets]['Bookie'].iteritems():
    bookie_index_list.setdefault(bookie, []).append(index)
        
for bookie, index_list in bookie_index_list.items():
    qualifying_bets_balance = sum(bet_spreadsheet.loc[index_list, 'Stake']   \
                            + bet_spreadsheet.loc[index_list, 'Return'])
    print("{} {}".format(bookie, qualifying_bets_balance))

Betfred 20.5
Coral 9.55
Skybet 12.0
William Hill 18.5
BetWay 56.1


#### Using .groupby('Bookie') to calculate the balances turns out to be less complicated.

In [7]:
def qualifying_bet_balances(spreadsheet):
    """Returns a series with the balance for each bookie due to all 
       settled qualifying bets."""
    
    winning_qualifying_bets = (spreadsheet['Bet Type'] == 'Qualifying')   \
                               & (spreadsheet['Bet Result'] == 'Win')
    
    qualifying_by_bookie = spreadsheet[winning_qualifying_bets].groupby('Bookie') 
    
    # Our original stake is returned with qualifying bets
    qualifying_balances = qualifying_by_bookie['Stake'].sum()   \
                             + qualifying_by_bookie['Return'].sum()
    
    # Smarkets balance will be calculated separately 
    if 'Smarkets' in qualifying_balances.index:
        qualifying_balances.drop('Smarkets', inplace=True)
        
    return qualifying_balances

qualifying_bet_balances(bet_spreadsheet)

Bookie
BetWay          56.10
Betfred         20.50
Coral            9.55
Skybet          12.00
William Hill    18.50
dtype: float64

In [8]:
def free_bet_balances(spreadsheet):
    """Returns a series with the balance for each bookie due to all 
       settled free bets."""

    winning_free_bets = (spreadsheet['Bet Type'] == 'Free (SNR)') \
                               & (spreadsheet['Bet Result'] == 'Win')
    
    free_by_bookie = spreadsheet[winning_free_bets].groupby('Bookie')
    
    # Our original state is not returned with free bets
    # SNR means "stake not returned"
    free_balances = free_by_bookie['Return'].sum()
    
    # Smarkets balance will be calculated separately 
    if 'Smarkets' in free_balances.index:
        free_balances.drop('Smarkets', inplace=True)
    
    return free_balances

free_bet_balances(bet_spreadsheet)

Bookie
Coral    110.0
Name: Return, dtype: float64

In [14]:
qualifying_bet_balances(bet_spreadsheet) + free_bet_balances(bet_spreadsheet)


Bookie
BetWay             NaN
Betfred            NaN
Coral           119.55
Skybet             NaN
William Hill       NaN
dtype: float64

#### Add series could be useful in future which is way the syntax is generalised.

In [12]:
def add_series(series_1, series_2):
    """Removes NaNs when two series are added together and some indices don't match.
       Indices which don't match keep values from original series.
       Still works if all indices from both series match."""
    
    potential_sum = series_1 + series_2
    
    removed_NaN_sum = potential_sum.fillna(series_1)  \
                                   .fillna(series_2)
    return removed_NaN_sum

add_series(qualifying_bet_balances(bet_spreadsheet), free_bet_balances(bet_spreadsheet))


Bookie
BetWay           56.10
Betfred          20.50
Coral           119.55
Skybet           12.00
William Hill     18.50
dtype: float64

In [13]:
def bookie_balances(spreadsheet):
    """Returns the total balance for each bookmaker for all settled bets."""
    
    return add_series(qualifying_bet_balances(spreadsheet), free_bet_balances(spreadsheet))

bookie_balances(bet_spreadsheet)

Bookie
BetWay           56.10
Betfred          20.50
Coral           119.55
Skybet           12.00
William Hill     18.50
dtype: float64

#### The generalised version of finding bookie_index_list from before

In [16]:
def value_index_list(series):
    """Iterates through values in a pandas series and returns a dictionary of the form
       {value : [indices for value]}

       Originally used to calculate qualifying bet balances before I realised that it was
       easier to use groupby instead.

       Could be useful in the future."""

    value_indices_dictionary = {}

    for index, value in series.iteritems():
        value_indices_dictionary.setdefault(value, []).append(index)

    return value_indices_dictionary

value_index_list(bet_spreadsheet['Bookie'])

{'Betfred': [0, 18],
 'Smarkets': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
 'Coral': [2, 4],
 'Skybet': [6, 8],
 'William Hill': [10, 12],
 'BetWay': [14, 16]}